# getDataFrame
!!!Make sure to have consistent naming for both the eeg and event files as well as equal number of eeg and event files!!!

In [1]:
import os
import sys
import time
import pandas as pd
import numpy as np

In [2]:
data_folder = 'data'
data_files = os.listdir(data_folder)
data_files = [i for i in data_files if '.csv' in i]
data_files_eeg = [i for i in data_files if 'eeg' in i]
data_files_event = [i for i in data_files if 'event' in i]
data_files_eeg.sort() # Make sure to have consistent naming for both the eeg and event files
data_files_event.sort()
data_eeg = [pd.read_csv('data/'+i) for i in data_files_eeg]
data_event = [pd.read_csv('data/'+i) for i in data_files_event]

In [3]:
for i in range(len(data_event)): # Convert event time into 4ms increment while matching with the eeg time
    adjusted_time = []
    for time in data_event[i]['timestamp']:
        index = data_eeg[i].loc[data_eeg[i]['timestamp'] <= time].index.values[-1] # Pick the latest eeg time before the event time
        adjusted_time.append(index * 4)
    data_event[i]['timestamp'] = adjusted_time
    data_event[i] = data_event[i].rename(columns={'timestamp' : 'time'})

for i in range(len(data_eeg)):# Convert eeg time into 4ms increment
    eeg = data_eeg[i]
    eeg['timestamp'] = list(range(0,len(eeg)*4,4))
    data_eeg[i] = data_eeg[i].rename(columns={'timestamp' : 'time'})
    data_eeg[i] = data_eeg[i].rename(columns={'4':'C4', '5':'C2', '6':'C1', '7':'C3'})

In [4]:
# Extract the labels from the events
event_start = [event.loc[event['event'].str.contains('start')] for event in data_event]
for es in event_start:
    es['event'] = [int(i[-1]) for i in np.array(es['event'])]
event_start_label = event_start
event_start_label = [esl.drop('time', axis = 1) for esl in event_start_label]

# Convert the labels into np arrays
labels = [esl.values.reshape(len(esl),) for esl in event_start_label]
labels = np.concatenate(labels, axis = None)

/Users/simonfei/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [7]:
for i in range(len(data_eeg)): # add 'EventStart' in eeg data
    es = [0] * len(data_eeg[i])
    sts = event_start[i]['time']
    for st in sts:
        es[round(st/4)] = 1
    data_eeg[i]['EventStart'] = es